In [152]:
import onnx
from onnx.shape_inference import infer_shapes
from onnx import helper
import os
print("ONNX version:", onnx.__version__)

ONNX version: 1.12.0


In [153]:
# Set some global variables
DESIRED_NODE = "Mul_"
ONNX_MODEL_FOLDER = "onnx_models"

In [154]:
# Load the baseline tinyyolo model
model = onnx.load("../tinyyolo_infershapes.onnx")
mulnodes = []
# Get the desired node 
for index,n in enumerate(model.graph.node):
    if DESIRED_NODE in n.name:
        mulnodes.append(index)

n1 = model.graph.node[mulnodes[1]-1]
# n2 = model.graph.node[mulnodes[1]]

def get_tensor_value_info(model, node):
    in_tensors = node.input
    print(in_tensors)
    out_tensor = node.output
    in_value_info = []
    out_value_info = []

    # Get the Valueprotos for the in and out tensors
    for x in model.graph.value_info:
        for i in in_tensors:
            if i in x.name:
                in_value_info.append(x)
            elif out_tensor[0] in x.name:
                out_value_info.append(x)
    in1 = in_value_info[0]
    in2 = in_value_info[1]
    out1 = out_value_info[0]
    return in_value_info, out_value_info

n1_in_val, n1_out_val = get_tensor_value_info(model,n1)
# n2_in_val, n2_out_val = get_tensor_value_info(model,n2)
in1,in2 = n1_in_val
# in3,in4 = n2_in_val

['280', '281']


In [155]:
import numpy as np
n1_tensor = np.random.random([16,3,3,3]).astype(np.int8)
# n2_tensor = np.random.random([1,16,1,1]).astype(np.float32)
n1_in2_init = onnx.helper.make_tensor(str(n1_in_val[1].name), onnx.TensorProto.INT8, n1_tensor.shape, vals = n1_tensor)
# n2_in2_init = onnx.helper.make_tensor(str(n2_in_val[1].name), onnx.TensorProto.FLOAT, n2_tensor.shape, vals = n2_tensor)
# init = onnx.helper.make_tensor(str(in2.name), onnx.TensorProto.FLOAT, b_tensor.shape, vals = b_tensor)
# np.save("n1_tensor", n1_tensor)
# np.save("n2_tensor", n2_tensor)

In [156]:
in1.name,in2.name,
# in3.name,in4.name

('280', '281')

In [157]:
# Create a new graph from the desired node. 
graph = helper.make_graph(
    nodes=[n1],
    name="simple_graph",
    inputs=[in1],
    initializer = [n1_in2_init],
    value_info=[in2],
    outputs=[n1_out_val[0]],
)

# Save the model to the onnx model folder
if not os.path.exists(ONNX_MODEL_FOLDER):
    os.makedirs(ONNX_MODEL_FOLDER)
onnx_model = helper.make_model(graph, producer_name="multiply-model")
onnx.save(onnx_model, ONNX_MODEL_FOLDER + '\multiply_model_split.onnx')